In [145]:
from yelpapi import YelpAPI
import json
import time as time
import numpy as np

f = open('.api_keys')
api_key = json.loads(f.read())['yelp_api']
yelp_api = YelpAPI(api_key)

In [106]:
import pandas as pd
import pyarrow.feather as feather


houses_df = feather.read_feather('data/dutchess_county_residence_sales_07_2019_06_2022.feather')

In [124]:
sample = houses_df.sample(200)

In [125]:
grocery = []
elementaryschools = []
highschools = [] 
preschools = []
coffee = [] 
hospitals = []
restaurants = []

search_terms = {
    'grocery': grocery, 
    'elementaryschools':elementaryschools, 
    'highschools':highschools, 
    'preschools':preschools, 
    'coffee':coffee, 
    'hospitals':hospitals,
    'restaurants':restaurants
}

count = 0

for index, house in sample.iterrows():
    long = house['long']
    lat = house['lat']
    for search_term in search_terms.keys():
        results = yelp_api.search_query(categories=search_term, longitude=long, latitude=lat, sort_by='distance', limit=50)
        search_terms[search_term].extend(results['businesses'])
    print(f'{count+1} / {len(sample)}', end='\r')
    count += 1
    

In [169]:
dfs = {x:pd.DataFrame(search_terms[x]).drop_duplicates(subset='id') for x in search_terms.keys()}

In [171]:
for key in dfs.keys():
    this_df = dfs[key]
    try:
        this_df = this_df.drop(columns=['price'])
    except:
        pass
    this_df = this_df[this_df['is_closed'] == False].drop(columns=['alias','image_url','is_closed','url','review_count','categories','transactions','location','phone','display_phone','distance'])
    this_df['rating'] = this_df['rating'].astype(float)
    this_df = this_df[this_df['rating'] > 3]
    this_df['long'] = np.nan
    this_df['lat'] = np.nan
    for index, row in this_df.iterrows():
        this_df.loc[index, 'long'] = row['coordinates']['longitude']
        this_df.loc[index, 'lat'] = row['coordinates']['latitude']
    this_df = this_df.drop(columns=['coordinates'])

    dfs[key] = this_df

In [219]:
for key in dfs.keys():
    feather.write_feather(dfs[key], f'data/dutchess_county_{key}_list.feather')